In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import pprint
import tqdm
import math
import datetime
from tensorflow.python.client import device_lib 
from matplotlib import pyplot as plt

print(device_lib.list_local_devices())


( train_ds , test_ds ) , ds_info = tfds . load ("mnist", split =["train", "test"], as_supervised = True , with_info = True )


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7662868370469732714
xla_global_id: -1
]


In [2]:

#tfds.show_examples(train_ds, ds_info)

# • How many training/test images are there?

print("length of train dataset: ", len(train_ds), "length of test dataset: ",  len(test_ds))

# • What’s the image shape?

# ds_info tells us: 'image': Image(shape=(28, 28, 1), dtype=uint8),

# • What range are pixel values in?

# not that efficient, but gives the correct answer of 255
"""best_max_value = 0
for elem in train_ds:
  max_val = np.max(elem[0].numpy())
  if max_val > best_max_value:
    best_max_value = max_val

print("highest value of pixel: ", best_max_value)"""

length of train dataset:  60000 length of test dataset:  10000


'best_max_value = 0\nfor elem in train_ds:\n  max_val = np.max(elem[0].numpy())\n  if max_val > best_max_value:\n    best_max_value = max_val\n\nprint("highest value of pixel: ", best_max_value)'

In [3]:
#ds_info

In [4]:
def preprocess_data(dataset):
  # 1. change datatype from uint8 into float values
  dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))

  # 2. flatten the image
  dataset = dataset.map(lambda img, target: (tf.reshape(img, (-1, )), target))

  # 3. normalize from 0 - 255 into -1 - 1
  dataset = dataset.map(lambda img, target: (img/128.0 - 1, target))

  # 4. one-hot encode the targets
  dataset = dataset.map(lambda img, target: (img, tf.one_hot(target, depth=10)))

  dataset = dataset.cache()
  dataset = dataset.shuffle(1000)
  dataset = dataset.batch(32)
  dataset = dataset.prefetch(28)
  return dataset

train_ds = train_ds.apply(preprocess_data)
test_ds = test_ds.apply(preprocess_data)

In [5]:
class MNIST_predictor(tf.keras.Model):
    
    def __init__(self):
        super(MNIST_predictor, self).__init__()
        self.layer_list = [
                              tf.keras.layers.Dense(256, activation=tf.nn.relu),
                              tf.keras.layers.Dense(256, activation=tf.nn.relu),
                              tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                          ]

    #@tf.function
    def call(self, x):
        for layer in self.layer_list:
          x = layer(x)
        return x

In [6]:
# • The number of epochs
EPOCHS = 10

# • The training dataset
# train_ds

# • The test dataset
# test_ds

# • The loss function
loss_fct = tf.keras.losses.CategoricalCrossentropy()

# • The optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.1)

# • The model object
model = MNIST_predictor()


# • Different arrays for the different values you want to track for visualization

In [7]:
#@tf.function
def test_step(x, y, model, loss_function):
    predictions = model(x, training=False)
    loss = loss_function(y, predictions) # why?  + tf.reduce_sum(self.losses)
    correct = np.argmax(predictions, axis=1) == np.argmax(y, axis=1)
    return loss, np.mean(correct)


#@tf.function
def train_step(x, y, model, loss_function, optimizer):
  with tf.GradientTape() as tape:
      predictions = model(x)
      loss = loss_function(y, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss



def train_loop(epochs,
               train_ds,
               test_ds,
               loss_function,
               optimizer,
               model):

    train_losses = []
    test_losses = []
    accuracys = []

    for epoch in range(epochs):
      count = 0
      print("epoch: ", epoch)
      train_loss = []
      test_loss = []
      acc = []
      
      # training
      for x, y in train_ds.take(1000):
        count += 1

        loss = train_step(x, y, model, loss_function, optimizer)
        train_loss.append(loss)

        if count % 10000 == 0:
           print(count)

      train_losses.append(tf.reduce_mean(train_loss).numpy())
      print("training loss: ", train_losses)

      # testing
      for x, y in test_ds.take(200):
        loss, correct = test_step(x, y, model, loss_function)
        test_loss.append(loss)
        acc.append(correct)

      accuracys.append(tf.reduce_mean(acc).numpy())
      test_losses.append(tf.reduce_mean(test_loss).numpy())
      print("test loss: ", test_losses, "\ntest_acc: ", accuracys, "\n")

    

In [8]:
train_loop(EPOCHS, train_ds, test_ds, loss_fct, opt, model)

epoch:  0


KeyboardInterrupt: 

In [ ]:
def visualization (train_losses , train_accuracies , test_losses ,test_accuracies):
    plt.figure ()
    line1 , = plt.plot(train_losses , "b-")
    line2 , = plt.plot(test_losses , "r-")
    line3 , = plt.plot(train_accuracies , "b:")
    line4 , = plt.plot(test_accuracies , "r:")
    plt . xlabel (" Training steps ")
    plt . ylabel (" Loss / Accuracy ")
    plt . legend (( line1 , line2 , line3 , line4 ) , (" training loss ", " testloss ", " train accuracy ", " test accuracy "))
    plt.show ()

visualization()